In [ ]:
import os
import h5py
import numpy as np
import matplotlib
matplotlib.rcParams['font.family'] = 'monospace'
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import yaml
from scipy import stats

# the directory with results
result_path = f"{os.curdir}/../outputs/2023-12-02/12-40-40"

result = h5py.File(f"{result_path}/result.hdf5", 'r')
with open(f"{result_path}/.hydra/config.yaml", "r") as file:
    config = yaml.safe_load(file)
plot_info = f'''
CONFIGURATION:

    general_parameters:

        spins_number:  {config["parameters"]["spins_number"]}
        gamma:         {config["parameters"]["gamma"]}
        tau:           {config["parameters"]["tau"]}
        ensemble_size: {config["parameters"]["ensemble_size"]}

    couplings_ensemble_parameters:

        density:       {config["couplings_ensemble"]["density"]}
        type:          {config["couplings_ensemble"]["type"]}
        parameters:    {config["couplings_ensemble"]["params"]}

    local_fields_ensemble_parameters:

        type:          {config["local_fields_ensemble"]["type"]}
        parameters:    {config["local_fields_ensemble"]["params"]}
'''
data_shape = (
    len(config["parameters"]["spins_number"]),
    len(config["parameters"]["gamma"]),
    len(config["parameters"]["tau"]),
)
mean_gaps = np.zeros(data_shape)
mean_sq_gaps = np.zeros(data_shape)
for sn, spins_number in enumerate(result.keys()):
    for gn, gamma in enumerate(result[spins_number].keys()):
        for tn, tau in enumerate(result[spins_number][gamma].keys()):
            mean_gap = (1. - np.abs(result[spins_number][gamma][tau]["relax"])).mean()
            mean_sq_gap = ((1. - np.abs(result[spins_number][gamma][tau]["relax"])) ** 2).mean()
            mean_gaps[sn, gn, tn] = mean_gap
            mean_sq_gaps[sn, gn, tn] = mean_sq_gap

mean_gaps_vs_spins_number = mean_gaps.mean((1, 2))
mean_sq_gaps_vs_spins_number = mean_sq_gaps.mean((1, 2))
std_gaps_vs_spins_number = np.sqrt(mean_sq_gaps_vs_spins_number - mean_gaps_vs_spins_number ** 2)

mean_gaps_vs_gamma = mean_gaps.mean((0, 2))
mean_sq_gaps_vs_gamma = mean_sq_gaps.mean((0, 2))
std_gaps_vs_gamma = np.sqrt(mean_sq_gaps_vs_gamma - mean_gaps_vs_gamma ** 2)

mean_gaps_vs_tau = mean_gaps.mean((0, 1))
mean_sq_gaps_vs_tau = mean_sq_gaps.mean((0, 1))
std_gaps_vs_tau = np.sqrt(mean_sq_gaps_vs_tau - mean_gaps_vs_tau ** 2)

spins_number = np.array([int(s) for s in config["parameters"]["spins_number"]])
gamma = np.array([float(g) for g in config["parameters"]["gamma"]])
tau = np.array([float(t) for t in config["parameters"]["tau"]])
slope, intercept, _, _, _ = stats.linregress(spins_number, np.log2(mean_gaps_vs_spins_number))
fit_x = np.linspace(spins_number[0], spins_number[-1], 100)
fit_y = 2 ** (fit_x * slope + intercept)
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3)
fig.text(0.07, 0.9, plot_info)
fig.set_size_inches(15, 10)
# ------------------------------------------------------
ax1.set_title("Averaged over rest of params")
ax1.plot(spins_number, mean_gaps_vs_spins_number, "ob-")
ax1.plot(fit_x, fit_y, "r--")
ax1.fill_between(
    spins_number,
    mean_gaps_vs_spins_number - std_gaps_vs_spins_number,
    mean_gaps_vs_spins_number + std_gaps_vs_spins_number,
    color='b', alpha=.1)
ax1.set_yscale("log")
ax1.set_ylabel("mean_gap")
ax1.set_xlabel("spins_number")
ax1.legend(["data", f"fit(k = {-slope})"])
# ------------------------------------------------------
ax2.set_title("Averaged over rest of params")
ax2.plot(gamma, mean_gaps_vs_gamma, "ob-")
ax2.fill_between(
    gamma,
    mean_gaps_vs_gamma - std_gaps_vs_gamma,
    mean_gaps_vs_gamma + std_gaps_vs_gamma,
    color='b', alpha=.1)
ax2.set_yscale("log")
ax2.set_xlabel("gamma")
# ------------------------------------------------------
ax3.set_title("Averaged over rest of params")
ax3.plot(tau, mean_gaps_vs_tau, "ob-")
ax3.fill_between(
    tau,
    mean_gaps_vs_tau - std_gaps_vs_tau,
    mean_gaps_vs_tau + std_gaps_vs_tau,
    color='b', alpha=.1)
ax3.set_yscale("log")
ax3.set_xlabel("tau")
ax3.set_xscale("log")
# ------------------------------------------------------
gamma_idx = int(len(gamma) / 2)
g = gamma[gamma_idx]
mg = mean_gaps[-1, gamma_idx]
msg = mean_sq_gaps[-1, gamma_idx]
std = np.sqrt(msg - mg ** 2)
ax4.set_title(f"spins_number: {spins_number[-1]}, gamma: {g}")
ax4.plot(tau, mg, color='b', label=f"tau={t}")
ax4.fill_between(
    tau,
    mg - std,
    mg + std,
    color='b', alpha=.1
)
ax4.set_yscale("log")
ax4.set_xscale("log")
ax4.set_xlabel("tau")
# ------------------------------------------------------
colors = cm.plasma(np.linspace(0, 1, len(tau)))
ax5.set_title(f"spins_number: {spins_number[-1]}")
for mg, msg, c, t in zip(mean_gaps[-1].T, mean_sq_gaps[-1].T, colors, tau):
    std = np.sqrt(msg - mg ** 2)
    ax5.plot(gamma, mg, color=c, label=f"tau={t}")
    ax5.fill_between(
        gamma,
        mg - std,
        mg + std,
        color=c, alpha=.1
    )
ax5.set_yscale("log")
ax5.set_xlabel("gamma")
ax5.set_ylabel("mean_gap")
ax5.legend()
# ------------------------------------------------------
colors = cm.plasma(np.linspace(0, 1, len(gamma)))
ax6.set_title(f"spins_number: {spins_number[-1]}")
for mg, msg, c, g in zip(mean_gaps[-1], mean_sq_gaps[-1], colors, gamma):
    std = np.sqrt(msg - mg ** 2)
    ax6.plot(tau, mg, color=c, label=f"gamma={g}")
    ax6.fill_between(
        tau,
        mg - std,
        mg + std,
        color=c, alpha=.1
    )
ax6.set_yscale("log")
ax6.set_xlabel("tau")
ax6.set_ylabel("mean_gap")
ax6.set_xscale("log")
ax6.legend()


In [ ]:
fig.savefig(f"{result_path}/plot.pdf", bbox_inches='tight')